In [1]:
import os
import sys
import time
import glob
import io
import pytesseract
from pytesseract import Output
from PIL import Image, ImageDraw
import cv2
from pathlib import Path
import pandas as pd
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None
import layoutparser

In [2]:
nb_dir = '/home/dhiraj/Documents/anuvaad/anuvaad-etl/anuvaad-extractor/block-merger/src'
sys.path.append(nb_dir)
sys.path.append(os.path.split(nb_dir)[0])

import config
import src.utilities.app_context as app_context
from src.utilities.primalaynet.infer import draw_box
app_context.init()
app_context.application_context = dict({'task_id': 'BM-DUMMY'})

#path to craft model, weight can be pulled form the production bracnh of repo
config.CRAFT_MODEL_PATH = "/home/dhiraj/Documents/anuvaad/anuvaad-etl/anuvaad-extractor/block-merger/src/utilities/craft_pytorch/model/craft_mlt_25k.pth"
config.CRAFT_REFINE_MODEL_PATH  = '/home/dhiraj/Documents/anuvaad/anuvaad-etl/anuvaad-extractor/block-merger/src/utilities/craft_pytorch/model/craft_refiner_CTW1500.pth' 


from src.services.main import generate_response,break_blocks,merge_vertically,merge_horizontally,extract_images_and_text_regions

Loading weights of refiner from checkpoint (./model/craft_refiner_CTW1500.pth)


In [3]:
import layoutparser as lp
model_primalaynet = lp.Detectron2LayoutModel('lp://PrimaLayout/mask_rcnn_R_50_FPN_3x/config',label_map = {1:"TextRegion", 2:"ImageRegion", 3:"TableRegion", 4:"MathsRegion", 5:"SeparatorRegion", 6:"OtherRegion"},extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.5])


In [4]:
def draw_box(filepath,desired_width, desired_height,save_dir, df, color="green", save=False):
    image  = Image.open(filepath)
    image  = image.resize((desired_width, desired_height))
    draw   = ImageDraw.Draw(image)
    if type(df) is list:
        for i in df:
            for index, row in i.iterrows():
                left   = int(row['text_left'])
                right  = int(row['text_width'] + left)
                top    = int(row['text_top'])
                bottom = int(row["text_height"] + top)
                draw.rectangle(((left, top), (right,bottom)), outline=color,width=3)
    else:

        for index, row in df.iterrows():
            left   = int(row['text_left'])
            right  = int(row['text_width'] + left)
            top    = int(row['text_top'])
            bottom = int(row["text_height"] + top)
            draw.rectangle(((left, top), (right,bottom)), outline=color,width=3)
    save_filepath = os.path.join(save_dir, os.path.basename(filepath))
    if save:
        image.save(save_filepath)
    
    return image

In [5]:
#dir which contains the pdf file
input_dir = '/home/dhiraj/Documents/Anuwad/anuvaad/anuvaad-etl/anuvaad-extractor/block-merger/src/notebooks/sample-data/input/'
filename = 'footer_test.pdf'
lang     = 'en'

In [6]:
[pdf_data,flags] = extract_images_and_text_regions(filename, input_dir,lang,page_layout='double_column')
[pdf_data,flags] = merge_horizontally([pdf_data,flags])
[pdf_data,flags] = merge_vertically([pdf_data,flags])
[pdf_data,flags] = break_blocks([pdf_data,flags])
#response        = generate_response([pdf_data,flags])

[2020-12-11 13:01:32,556] {loghandler.py:21} MainThread INFO in loghandler: document preprocessing started ===>
[2020-12-11 13:01:32,559] {loghandler.py:21} MainThread INFO in loghandler: created processing directories successfully /home/dhiraj/Documents/Anuwad/anuvaad/anuvaad-etl/anuvaad-extractor/block-merger/src/notebooks/sample-data/input/footer_test_e437e6a2-3b82-11eb-8d8f-f3bdb2363650
[2020-12-11 13:01:32,560] {loghandler.py:21} MainThread INFO in loghandler: filepath /home/dhiraj/Documents/Anuwad/anuvaad/anuvaad-etl/anuvaad-extractor/block-merger/src/notebooks/sample-data/input/footer_test.pdf, working_dir /home/dhiraj/Documents/Anuwad/anuvaad/anuvaad-etl/anuvaad-extractor/block-merger/src/notebooks/sample-data/input/footer_test_e437e6a2-3b82-11eb-8d8f-f3bdb2363650
[2020-12-11 13:01:33,177] {loghandler.py:21} MainThread INFO in loghandler: Extracting xml of /home/dhiraj/Documents/Anuwad/anuvaad/anuvaad-etl/anuvaad-extractor/block-merger/src/notebooks/sample-data/input/footer_tes

In [7]:
# width_ratio = pdf_data['page_width'] / pdf_data['pdf_image_width']
# height_ratio = pdf_data['page_height'] / pdf_data['pdf_image_height']
# predict_primanet(img_filepath,pdf_bg_img_filepaths,to_check[0],width_ratio,height_ratio)

In [8]:
page_index = 0
to_check        = pdf_data['p_dfs']
#to_check_in_df        = pdf_data['v_dfs'][page_index]
img_filepath    = pdf_data['pdf_image_paths'][page_index]
pdf_bg_img_filepaths  = pdf_data['pdf_bg_img_filepaths'][page_index]
df              = to_check[page_index]

desired_width, desired_height = pdf_data['page_width'] , pdf_data['page_height']
draw_box(img_filepath,desired_width, desired_height, 'path',df, color="red", save=False)

In [9]:
df

index xml_index text_top text_left text_width text_height  \
0    NaN         0     1176       149        287          14   
1    NaN         1     1176       747         83          14   
2    NaN         2       89       629        197          19   
3    NaN         3      165       298        379          69   
4    NaN         5      266       270        435          19   
5    NaN         7      312       578        249          23   
6    NaN         6      316       149         68          19   
7    NaN         8      366       365         89          19   
8    NaN        10      412       577        250          23   
9    NaN         9      416       149        301          19   
10    11        11      520       377        223          26   
11    12        12      578       149        200          19   
12   NaN        13      625       203        623         298   
13   NaN        23      974       149        678         160   

                                                 text font_size font_family  \
0             Criminal Appeal Nos. 2267-2268 of 2009         14       Arial   
1                                        Page 1 of 4         14       Arial   
2                                     NON-REPORTABLE         20       Arial   
3    IN THE SUPREME COURT OF INDIA  CRIMINAL APPEL...        20       Arial   
4             CRIMINAL APPEAL NOS. 2267-2268 OF 2009         20       Arial   
5                        …..            APPELLANT(S)         20     ArialMT   
6                                              UDIYA         20     ArialMT   
7                                             VERSUS         20     ArialMT   
8                           …..        RESPONDENT(S)         20     ArialMT   
9                            STATE OF MADHYA PRADESH         20     ArialMT   
10                                   J U D G M E N T         28       Arial   
11                                 SANJIV KHANNA, J.         20       Arial   
12   By  the  impugned  judgment  dated  July  07,...        20     ArialMT   
13   2.  Having considered the testimony of Jeevni...        20     ArialMT   

   font_color  attrib font_family_updated font_size_updated  \
0     #000000  FOOTER               Arial                14   
1     #000000  FOOTER               Arial                14   
2     #000000    BOLD               Arial                20   
3     #000000    BOLD               Arial                20   
4     #000000    BOLD               Arial                20   
5     #000000                     ArialMT                20   
6     #000000                     ArialMT                20   
7     #000000                     ArialMT                20   
8     #000000                     ArialMT                20   
9     #000000                     ArialMT                20   
10    #000000    BOLD               Arial                28   
11    #000000    BOLD               Arial                20   
12    #000000                     ArialMT                20   
13    #000000                     ArialMT                20   

                                             children level_0 avg_line_height  
0                                                None     NaN              14  
1                                                None     NaN              14  
2                                                None     NaN              19  
3   {"level_0":{"1":3,"2":4},"index":{"1":3,"2":4}...     NaN              19  
4                                                None     NaN              19  
5                                                None     NaN              23  
6                                                None     NaN              19  
7                                                None     NaN              19  
8                                                None     NaN              23  
9                                                None     NaN              19  
10                            

In [10]:
for i in to_check_in_df['children']:
    d =pd.read_json(i)
    print(d)
    print(d['children'])

NameError: name 'to_check_in_df' is not defined

In [ ]:
import numpy as np
p_df_image = draw_box(img_filepath,desired_width, desired_height, 'path',df, color="red", save=False)
sc_pdf_image = draw_box(img_filepath,desired_width, desired_height, 'path',to_check_in_df, color="red", save=False)
stiched_imag = np.hstack([np.array(p_df_image) ,np.array(sc_pdf_image)])
compare_image = Image.fromarray(stiched_imag)
compare_image.save("/home/naresh/1.jpg")

In [ ]:
def predict_primanet(image_path,bg_image_path,in_df,width_ratio,height_ratio):
    image1  = cv2.imread(image_path)
    image2  = cv2.imread(bg_image_path)
    image = image1[..., ::-1] 
    layout = model_primalaynet.detect(image)
    boxes,coords = draw_box(layout, in_df,width_ratio,height_ratio)
    final_coord= []
    for coord in coords:
        temp_dict={}
        temp_dict['text_left']=coord[0]; temp_dict['text_top']=coord[1]
        temp_dict['text_width']=coord[2]; temp_dict['text_height']=coord[3]
        #draw.rectangle(coord, width=3,outline='red')
        final_coord.append(temp_dict)
        cv2.rectangle(image1, (int(coord[0]),int(coord[1])), (int(coord[2]),int(coord[3])),(255,255,0),3)
        cv2.rectangle(image2, (int(coord[0]*width_ratio),int(coord[1]*height_ratio)), (int(coord[2]*width_ratio),int(coord[3]*height_ratio)),(255,255,0),3)

        cv2.imwrite("1.jpg",image1)
        cv2.imwrite("2.jpg",image2)

    return final_coord